<a href="https://colab.research.google.com/github/omidbazgirTTU/LLMs/blob/main/PPO_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proximal Policy Implementation (PPO)
## Author Omid Bazgir
## Thanks for the great tutorial by Ehsan Kamalinejad (EK)
### Linke to the tutorial on YouTube https://www.youtube.com/watch?v=3uvnoVjM8nY&list=PLb9xatikqn0fwsS-Le1mkyQ2uZzK8DeP1&index=3

### Basic implementation of PPO (this example is for cart pole program) which is a reinforcement learning (RL) algorithm with many applications including game design, development, large language models (LLMs). PPO is being used in LLMs as as finetuning technique through reinforcement learning with human feedback (RLHF).

### more details on the cart pole problem including the how to define the reward values, action space and so on is provided in the Gymnasium documentation https://www.gymlibrary.dev/environments/classic_control/cart_pole/

References introduced by EK:
- [EK's Video Lecture](https://www.youtube.com/watch?v=3uvnoVjM8nY) This is the lecture where we did a deep dive into the theory of PPO.
- [OpenAI PPO Repo](https://github.com/openai/baselines/blob/master/baselines/ppo2/runner.py) This is helpful as a reference for further implementations.
- [PPO Paper](https://arxiv.org/abs/1707.06347) This is the original paper that introduced PPO.
- [Sergey Levine UC Berkley CS285](http://rail.eecs.berkeley.edu/deeprlcourse/) This is a complete course in RL.
- [Pieter Abbeel mini-course](https://www.youtube.com/watch?v=2GwBez0D20A&list=PLwRJQ4m4UJjNymuBM9RdmB3Z9N5-0IlY0) This is a mini-course focusing on TRPO, PPO, DDPG and model free RL.
- [OpenAI Documentation on RL](https://spinningup.openai.com/en/latest/index.html) THis is OpenAI documentation on RL and parts of our code was borrowed from here.
- [labml.ai](https://nn.labml.ai/) This repo contains popular papers with their annotated PyTorch implementations.
- [cleanrl](https://github.com/vwxyzjn/cleanrl) This repo has clean implementations of RL algorithms and parts of our code was borrowed from here.



In [1]:
# installing dependencies
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
!pip install moviepy omegaconf matplotlib
!pip install gym==0.26.2
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests
!pip install gym[classic_control] gym[atari] gym[accept-rom-license] gym[other]

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=1b9361442a3188671fbfe6416f48cc1b2a945a115ee3d6a849764f9a70966a08
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827620 sha256=2fbe11ecc1f57d257aebbadb5ce4ed3f13f7e963c3d4f1fdd1c035ba539605e1
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.6 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.
  Cloning https://github.com/carlosluis/stable-baselines3 (to revision fix_tests) to /tmp/pip-req-buil

In [20]:
import time
import random
import numpy as np
import matplotlib.pylab as plt
plt.style.use('dark_background')
from tqdm.notebook import tqdm
from omegaconf import DictConfig

import gym

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.distributions.categorical import Categorical

from IPython.display import Video

In [21]:
# set up
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [23]:
configs = {
    # experiment arguments
    "exp_name": "cartpole",
    "gym_id": "CartPole-v1", # the id of from OpenAI gym
    # training arguments
    "learning_rate": 1e-3, # the learning rate of the optimizer
    "total_timesteps": 1000000, # total timesteps of the training
    "max_grad_norm": 0.5, # the maximum norm allowed for the gradient
    # PPO parameters
    "num_trajcts": 32, # N
    "max_trajects_length": 64, # T
    "gamma": 0.99, # gamma
    "gae_lambda":0.95, # lambda for the generalized advantage estimation
    "num_minibatches": 2, # number of mibibatches used in each gradient
    "update_epochs": 2, # number of full rollout storage creations
    "clip_epsilon": 0.2, # the surrogate clipping coefficient
    "ent_coef": 0.01, # entroy coefficient controlling the exploration factor C2
    "vf_coef": 0.5, # value function controlling value estimation importance C1
    # visualization and print parameters
    "num_returns_to_average": 3, # how many episodes to use for printing average return
    "num_episodes_to_average": 23, # how many episodes to use for smoothing of the return diagram
    }

# batch_size is the size of the flatten sequences when trajcts are flatten
configs['batch_size'] = int(configs['num_trajcts'] * configs['max_trajects_length'])
# number of samples used in each gradient
configs['minibatch_size'] = int(configs['batch_size'] // configs['num_minibatches'])

configs = DictConfig(configs)

run_name = f"{configs.gym_id}__{configs.exp_name}__{seed}__{int(time.time())}"

## ENV
`envs` us set of parallel environments each holding a random initiali `state` and accepts an `action` to change and return its new state.

In [24]:
# creating an env with random state
def make_env_func(gym_id, seed, idx, run_name, capture_video = False):
  def env_fun():
    env = gym.make(gym_id, render_mode = "rgb_gray")
    env = gym.wrappers.RecordEpisodeStatistics(env)
    if capture_video:
      # initiate the video capture if not already initiated
      if idx ==0:
        #wrapper to create the video of the performance
        env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
    env.action_space.seed(seed)
    env.observation_space.seed(seed)

    return env
  return env_fun

In [25]:
# create N (here is 32) parallel envs
envs = []
for i in range(configs.num_trajcts):
  envs.append(make_env_func(configs.gym_id, seed+i, i, run_name))
envs = gym.vector.SyncVectorEnv(envs)
envs

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:623: UserWarning: WARN: The environment is being initialised with mode (rgb_gray) that is not in the possible render_modes (['human', 'rgb_array']).
  logger.warn(


SyncVectorEnv(32)

## Model
A simple MLP (or fully connected layers FC) model that gets a state and has two methods:
* `agent.value_func(state)` gets a state and returns the estimated expected total feature rewards from that state $V_{\theta}(s)$.

* `agent.policy(state)` gets a state and returns next `action`, `log_prob` of actions, the `entropy` and `value`.

In [42]:
class FCBlock(nn.Module):
    """A generic fully connected residual block with good setup"""
    def __init__(self, embed_dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.GELU(),
            nn.Linear(embed_dim, 4*embed_dim),
            nn.GELU(),
            nn.Linear(4*embed_dim, embed_dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return x + self.block(x)

class Agent(nn.Module):
  """ an agent that creates actions and estimates values"""
  def __init__(self, env_observation_dim, action_space_dim, embed_dim = 64, num_blocks=2):
    super().__init__()
    # getting the observation and embed that into another space `embed_dim`
    self.embedding_layer = nn.Linear(env_observation_dim, embed_dim)
    # layers that are shared between policy head and value head
    # it not necessarily needed to have a shared layer, but here since that value and policy tasks are quite similar
    # we can use several shared layer to do multi-task learning
    self.shared_layers = nn.Sequential(*[FCBlock(embed_dim=embed_dim) for _ in range(num_blocks)])
    self.value_head = nn.Linear(embed_dim, 1)
    self.policy_head = nn.Linear(embed_dim, action_space_dim)
    # orthogonal initialization with a hi entropy for exploration at the start
    torch.nn.init.orthogonal_(self.policy_head.weight, 0.01)

  def value_func(self,state):
    hidden = self.shared_layers(self.embedding_layer(state))
    value = self.value_head(hidden)
    return value

  def policy(self, state, action=None):
    # plicy is supposed to create actions but here it takes actions as the input
    # this is for phase 2 of PPO where we want to analze the actions
    hidden = self.shared_layers(self.embedding_layer(state))
    logits = self.policy_head(hidden)
    # Pytoch categorical class for sampling and probability calcucation
    probs = Categorical(logits=logits)
    if action is None:
      action = probs.sample()
    return action, probs.log_prob(action), probs.entropy(), self.value_head(hidden)

## Generalized Advantage Estimation (GAE)

### $Advantage (s, a)$ calculates how better or worse the return of taking the action $a$ at the state $s$ is compared to expected return for all other actions in that state.

### we can approximate that with the below reverse formulas

$δ_{t} = r_{t} + γV(s_{t+1}) - V(s_{t})$

$\hat{A_{t}} = δ_{t} + γλ\hat{A}_{t+1}$


In [43]:
def gae(
    cur_observations,   # the current state when advantages will be calculated
    rewards,            # rewards collected from trajectories of shape [num_trajcts, max_trajcts_length]
    dones,              # binary marker of end of trajectories of shape [num_trajcts, max_trajcts_length]
    values,             # value estimates collected over trajectories of shape [num_trajcts, max_trajcts_length]
):

  advantages = torch.zeros((configs.num_trajcts, configs.max_trajects_length))
  last_advantage = 0

  # the value after the last step
  with torch.no_grad():
    last_value = agent.value_func(cur_observations).reshape(1,-1)

  # reverse recursive to calculate advantages based on the delta formula
  for t in reversed(range(configs.max_trajects_length)):
    # mask if episode completed after step t
    mask = 1.0 - dones[:,t] # --> if we are looking for those trajectories that were ended quicker we don't need to any further calculation so we use the variable mask
    last_value = last_value * mask
    last_advantage = last_advantage * mask
    delta = rewards[:,t] + configs.gamma * last_value - values[:,t]
    last_advantage = delta + configs.gamma * configs.gae_lambda * last_advantage
    advantages[:,t] = last_advantage
    last_value = values[:,t]

  advantages = advantages.to(device)
  returns = advantages + values

  return advantages, returns

## Creating Rollout stage

### Phase 1: rollout creation

1. Generate $N$ trajectories of length $T$ by $\pi_{θ_{old}}$
2. calculate $logits_{old}$ for the actions
3. calculate $V_{theta}$ along the trajectories
4. calculate advantage estimates $\hat{A}_{t}$
5. create a storage and add all items to it

In [49]:
def create_rollout(
    envs,               # parallel envs creating trajectories
    cur_observation,    # starting observation of shape [num_trajcts, observation_dim]
    cur_done,           # current termination status of shape [num_trajcts,]
    all_returns         # a list to track returns
):
  """
  rollout phase: create parallel trajectories and store them in the rolout storage
  """

  # cache empty tensors to store the rollouts
  observations = torch.zeros((configs.num_trajcts, configs.max_trajects_length) +
                             envs.single_observation_space.shape).to(device)
  actions = torch.zeros((configs.num_trajcts, configs.max_trajects_length) +
                        envs.single_action_space.shape).to(device)
  logprobs = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)
  rewards = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)
  dones = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)
  values = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)

  for t in range(configs.max_trajects_length):
    observations[:,t] = cur_observation
    dones[:,t] = cur_done

    # give observation to the model and collect action, logprobs of actions, entropy and value
    with torch.no_grad():
      action, logprob, entropy, value = agent.policy(cur_observation)

    values[:,t] = value.flatten()
    actions[:,t] = action
    logprobs[:,t] = logprob

    # apply the action to the env and collect observation and reward
    cur_observation, reward, cur_done, _, info = envs.step(action.cpu().numpy())
    rewards[:,t] = torch.Tensor(reward).to(device).view(-1)
    cur_observation = torch.Tensor(cur_observation).to(device)
    cur_done = torch.Tensor(cur_done).to(device)

    # if an episode ended store its total reward for progress report
    if info:
      for item in info['final_info']:
        if item and "episode" in item.keys():
          all_returns.append(item['episode']['r'])
          break

    # create rollout storage
    rollout = {
        'cur_observation': cur_observation,
        'cur_done': cur_done,
        'observations': observations,
        'actions': actions,
        'logprobs' : logprobs,
        'values' : values,
        'dones' : dones,
        'rewards' : rewards
    }

    return rollout


In [50]:
# creating a standard pytorch dataset to store the RL model output in a standard dataset to be used for downstream tasks (NLP, LLM)
class Storage(Dataset):
  def __init__(self, rollout, advantages, returns, envs):
    # fill in the storage and flatten the parallel trajectories
    self.observations = rollout['observations'].reshape((-1,) + envs.single_observation_space.shape)
    self.logprobs = rollout['logprobs'].reshape(-1)
    self.actions = rollout['actions'].reshape((-1,) + envs.single_observation_space.shape).long()
    self.advantages = advantages.reshape(-1)
    self.returns = returns.reshape(-1)


  def __getitem__(self, ix: int):
    item = [
        self.observations[ix],
        self.logprobs[ix],
        self.actions[ix],
        self.advantages[ix],
        self.returns[ix],
    ]
    return item

  def __len__(self) -> int:
    return len(self.observations)

## Loss Functions

$L^{CLIP} = min(\frac{\pi_{theta}(a_{t}|s_{t})}{\pi_{theta_{old}}(a_{t}|s_{t})}.\hat{A_{t}},
clip(\frac{\pi_{theta}(a_{t}|s_{t})}{\pi_{theta_{old}}(a_{t}|s_{t})},1-ϵ, 1+ϵ).\hat{A_{t}})$

$L^{VF} = \frac{1}{2} \left \lVert V_{\theta}(s) - (Σ_{t=0}^{T} γ^{t}r_{t}|s_{0}=s) \right \rVert_2^2 $

$L^{ENT} = entropy(\pi_{θ}(.|s_{t}))$

In [51]:
def loss_clip(
    mb_oldlogprob,   # old logprob of mini batch actions collected during the rollout
    mb_newlogprob,   # new logprob of mini batch actions created by the new policy
    mb_advantages,   # mini batch of advantages collected during the rolout

):
  """
  policy loss with clipping to control gradients
  """

  ratio = torch.exp(mb_newlogprob - mb_oldlogprob)
  ploicy_loss = -mb_advantages * ratio          # since here we want to maximize the reward and pytorch optimizer always look for minimum value, we convert the sign by multiplying the negative -1
  # clipped policy gradient loss enforces closeness
  clipped_loss = -mb_advantages * torch.clamp(ratio, 1 - configs.clip_epsilon, 1 + configs.clip_epsilon)
  pessimistic_loss = torch.max(ploicy_loss, clipped_loss).mean()
  return pessimistic_loss

def loss_vf(
    mb_oldreturns,    # mini batch of old returns collected during the rollout
    mb_newvalues,     # mini batch of values calculated by the new value function
):
  """
  enforcing the value function to give more accurate estimates of returns
  """
  mb_newvalues = mb_newvalues.view(-1)
  loss = 0.5 * ((mb_newvalues - mb_oldreturns)**2).mean()
  return loss

# Training

## The training loop has two phases, phase I and phase II

### Phase I: rollout creation

1. Generate $N$ trajectories of length $T$ by $\pi_{θ_{old}}$
2. calculate $logits_{old}$ for the actions
3. calculate $V_{theta}$ along the trajectories
4. calculate advantage estimates $\hat{A}_{t}$
5. create a storage and add all items to it

### Phase II: model update

- for k epochs
  1. select a minibatch of size $ M \le N*T $ from the storage
  2. Generate $logits$ along the minibatch with $\pi_{\theta}$
  3. Calculate average of $L^{SUR} = L^{CLIP} + c_{1}*L_{VF} + c_{2}*L^{ENT}$
- Update $\theta_{old} <--  \theta$

In [52]:
agent = Agent(
    env_observation_dim=envs.single_observation_space.shape[0],
    action_space_dim=envs.single_action_space.n
).to(device)

optimizer = optim.Adam(agent.parameters(), lr=configs.learning_rate)

In [54]:
# track returns
all_returns = []

# initialize the game
cur_observation = torch.Tensor(envs.reset()[0]).to(device)
cur_done = torch.zeros(configs.num_trajcts).to(device)

# progress bar
num_updates = configs.total_timesteps // configs.batch_size
progress_bar = tqdm(total=num_updates)

for update in range(1, num_updates + 1):


    ##############################################
    # Phase 1: rollout creation

    # parallel envs creating trajectories
    rollout = create_rollout(envs, cur_observation, cur_done, all_returns)

    cur_done = rollout['cur_done']
    cur_observation = rollout['cur_observation']
    rewards = rollout['rewards']
    dones = rollout['dones']
    values = rollout['values']

    # calculating advantages
    advantages, returns = gae(cur_observation, rewards, dones, values)

    # a dataset containing the rollouts
    dataset = Storage(rollout, advantages, returns, envs)

    # a standard dataloader made out of current storage
    trainloader = DataLoader(dataset, batch_size=configs.minibatch_size, shuffle=True)


    ##############################################
    # Phase 2: model update

    # linearly shrink the lr from the initial lr to zero
    frac = 1.0 - (update - 1.0) / num_updates
    optimizer.param_groups[0]["lr"] = frac * configs.learning_rate

    # training loop
    for epoch in range(configs.update_epochs):
        for batch in trainloader:
            mb_observations, mb_logprobs, mb_actions, mb_advantages, mb_returns = batch

            # we calculate the distribution of actions through the updated model revisiting the old trajectories
            _, mb_newlogprob, mb_entropy, mb_newvalues = agent.policy(mb_observations, mb_actions)

            policy_loss = loss_clip(mb_logprobs, mb_newlogprob, mb_advantages)

            value_loss = loss_vf(mb_returns, mb_newvalues)

            # average entory of the action space
            entropy_loss = mb_entropy.mean()

            # full weighted loss
            loss = policy_loss - configs.ent_coef * entropy_loss + configs.vf_coef * value_loss

            optimizer.zero_grad()
            loss.backward()

            # extra clipping of the gradients to avoid overshoots
            nn.utils.clip_grad_norm_(agent.parameters(), configs.max_grad_norm)
            optimizer.step()

    # progress bar
    if len(all_returns) > configs.num_returns_to_average:
        progress_bar.set_description(f"episode return: {np.mean(all_returns[-configs.num_returns_to_average:]):.2f}")
        progress_bar.refresh()
        progress_bar.update()

envs.close()

  0%|          | 0/488 [00:00<?, ?it/s]

IndexError: ignored